In [ ]:
#We will be using a Deep Learning Model because we have 5 Classifications (1-5 star yelp ratings) and whether those correlate to a 1-3 Michelin Star resturant.

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
michelin_star_df = pd.read_csv("michelin_star_data_cleaned.csv") # this will be the cleaned data from a previous step
michelin_star_df.head()

In [ ]:
# Drop any non-beneficial columns.
michelin_star_df = application_df.drop(columns =['XXXX','YYYY','ZZZZ'])

In [ ]:
# Determine the number of unique values in each column.
michelin_star_df.nunique()

In [ ]:
# Look at Yelp Star Review value counts for binning
yelp_counts = michelin_star_df.YELPSTARREVIEWS.value_counts()
yelp_counts

In [ ]:
# Visualize the value counts of YELPSTARREVIEWS
yelp_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...? May need to write a for loop in the case that the numbers are between whole numbers (4.6,etc.)
replace_yelp = list(yelp_counts[yelp_counts < 528].index)

# Replace in dataframe
for app in replace_yelp:
    michelin_star_df.YELPSTARREVIEWS = michelin_star_df.YELPSTARREVIEWS.replace(app,"Other")
    
# Check to make sure binning was successful
michelin_star_df.YELPSTARREVIEWS.value_counts()

In [ ]:
# Look at MICHELINSTARS value counts for binning
michelinstars_counts = michelin_star_df.MICHELINSTARS.value_counts()
michelinstars_counts

In [ ]:
# Visualize the value counts of MICHELINSTARS
michelinstars_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..? # Same as before. THis may not be necessary. 
replace_michelin = list(michelinstars_counts[michelinstars_counts < 1883].index)

# Replace in dataframe
for cls in replace_michelin:
   michelin_star_df.MICHELINSTARS = michelin_star_df.MICHELINSTARS.replace(cls,"Other")
    
# Check to make sure binning was successful
michelin_star_df.MICHELINSTARS.value_counts()

In [ ]:
# Generate our categorical variable lists
michelin_cat = michelin_star_df.dtypes[michelin_star_df.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(michelin_star_df[michelin_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(michelin_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
michelin_star_df = michelin_star_df.merge(encode_df,left_index=True, right_index=True)
michelin_star_df = michelin_star_df.drop(michelin_cat,1)
michelin_star_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = michelin_star_df['XXXXX'].values
X = michelin_star_df.drop(['XXXXX'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}hdf5"

In [ ]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")